In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np 


In [3]:
trainData = pd.read_csv("/content/drive/MyDrive/NLP/train.csv")
testData = pd.read_csv("/content/drive/MyDrive/NLP/test.csv")
del trainData["Id"]
del testData["Id"]

In [4]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import nltk

from nltk.corpus import movie_reviews
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')
import random
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from stop_words import get_stop_words

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
pronuns = ["a","about","above","after","again","against","all","am","an","and","any","are","aren’t","as",
           "at","be","because","been","before","being","below","between","both","but","by","can’t","cannot"
           ,"could","couldn’t","did","didn’t","do","does","doesn’t","doing","don’t","down","during","each",
           "few","for","from","further","had","hadn’t","has","hasn’t","have","haven’t","having","he","he’d"
           ,"he’ll","he’s","her","here","here’s","hers","herself","him","himself","his","how","how’s","i"
           ,"i’d","i’ll","i’m","i’ve","if","in","into","is","isn’t","it","it’s","its","itself","let’s","me"
           ,"more","most","mustn’t","my","myself","no","nor","not","of","off","on","once","only","or","other"
           ,"ought","our","oursourselves","out","over","own","same","shan’t","she","she’d","she’ll","she’s"
           ,"should","shouldn’t","so","some","such","than","that","that’s","the","their","theirs","them"
           ,"themselves","then","there","there’s","these","they","they’d","they’ll","they’re","they’ve"
           ,"this","those","through","to","too","under","until","up","very","was","wasn’t","we","we’d"
           ,"we’ll","we’re","we’ve","were","weren’t","what","what’s","when","when’s","where","where’s"
           ,"which","while","who","who’s","whom","why","why’s","with","won’t","would","wouldn’t","you"
           ,"you’d","you’ll","you’re","you’ve","your","yours","yourself","yourselves"]
pronuns_slang = ["a","about","above","after","again","against","all","am","an","and","any"
                ,"are","arent","as","at","be","because","been","before","being","below","between","both"
                ,"but","by","cant","cannot","could","couldnt","did","didnt","do","does","doesnt","doing"
                ,"dont","down","during","each","few","for","from","further","had","hadnt","has","hasnt"
                ,"have","havent","having","he","hed","hell","hes","her","here","heres","hers","herself"
                ,"him","himself","his","how","hows","i","id","ill","im","ive","if","in","into","is","isnt"
                ,"it","its","its","itself","lets","me","more","most","mustnt","my","myself","no","nor","not"
                ,"of","off","on","once","only","or","other","ought","our","oursourselves","out","over","own"
                ,"same","shant","she","shed","shell","shes","should","shouldnt","so","some","such","than"
                ,"that","thats","the","their","theirs","them","themselves","then","there","theres","these"
                ,"they","theyd","theyll","theyre","theyve","this","those","through","to","too","under","until"
                ,"up","very","was","wasnt","we","wed","well","were","weve","were","werent","what","whats","when"
                ,"whens","where","wheres","which","while","who","whos","whom","why","whys","with","wont","would",
                "wouldnt","you","youd","youll","youre","youve","your","yours","yourself","yourselves"]


pronuns = list(set(pronuns + pronuns_slang))

stw = list(get_stop_words('en')) + ['.',',',';','/','-','_',"a"] + list(stopwords.words('english')) + pronuns



In [7]:


import string


trainData['comment_without_sw'] = trainData['Comment'].apply(lambda x: ' '.join([word.translate(str.maketrans('', '', string.punctuation)).strip().lower() for word in x.split() 
if word.translate(str.maketrans('', '', string.punctuation)).strip().lower() not in (stw)]))

In [246]:
testData['comment_without_sw'] = testData['Comment'].apply(lambda x: ' '.join([word.translate(str.maketrans('', '', string.punctuation)).strip().lower() for word in x.split() 
if word.translate(str.maketrans('', '', string.punctuation)).strip().lower() not in (stw)]))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(trainData['comment_without_sw'])

In [9]:


x.toarray().shape


(8695, 20060)

In [10]:
import math
freq = pd.Series(' '.join(trainData.comment_without_sw).split()).value_counts()
N = 5
freq["things"]
size = len(trainData)
freq = freq[N < freq]
freq = freq[freq < 85]

freq

protein            84
concentration      84
times              84
low                84
higher             84
                   ..
replies             6
straightforward     6
shorts              6
tissues             6
wording             6
Length: 3418, dtype: int64

In [11]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

In [12]:
import gensim.downloader as api
path = api.load("glove-wiki-gigaword-50", return_path=True)
print(path)

/root/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz


In [13]:
model = api.load("glove-wiki-gigaword-50")

In [14]:
vector = model["einstein"]
approximate_neighbors = model.most_similar([vector], topn=11)

print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

Approximate Neighbors
('einstein', 1.0)
('relativity', 0.7438952922821045)
('bohr', 0.7286733388900757)
('physics', 0.7018452882766724)
('heisenberg', 0.7005619406700134)
('freud', 0.6971668601036072)
('theory', 0.6945189833641052)
('leibniz', 0.6941564679145813)
('mathematical', 0.6708714365959167)
('invented', 0.6495018601417542)
('goethe', 0.6423683166503906)


In [15]:
freq

protein            84
concentration      84
times              84
low                84
higher             84
                   ..
replies             6
straightforward     6
shorts              6
tissues             6
wording             6
Length: 3418, dtype: int64

In [16]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from gensim.similarities.index import AnnoyIndexer

# 100 trees are being used in this example
annoy_index = AnnoyIndexer(model, 5)

/usr/local/lib/python3.7/dist-packages/gensim/similarities/index.py:180: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


In [18]:
approximate_neighbors = model.most_similar("cat", topn=11, indexer=annoy_index)
# Neatly print the approximate_neighbors and their corresponding cosine similarity values
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

Approximate Neighbors
('cat', 1.0)
('dog', 0.8022635728120804)
('rabbit', 0.7250292301177979)
('cats', 0.6732944548130035)
('dogs', 0.6680222451686859)
('pig', 0.6426196098327637)
('bug', 0.6370916366577148)
('elephant', 0.636787474155426)
('dragon', 0.6212137043476105)
('duck', 0.6148057579994202)
('horse', 0.6138353645801544)


In [19]:

freq_sub = freq
for key in freq_sub.keys():
  if not key in model.vocab:
    continue
  for similiar in model.most_similar([model[key]], topn=11,indexer=annoy_index):
    if (similiar[0] != key) and (similiar[0] in freq_sub.keys()):
      freq_sub = freq_sub.drop(similiar[0])

1 for 67 mb model

5:30 for 257 mb model

In [20]:
freq_sub

mrna               79
person             73
deleted            70
universe           65
amino              62
                   ..
ii                  6
replies             6
straightforward     6
shorts              6
wording             6
Length: 691, dtype: int64

In [21]:
words = model.most_similar([model[key]], topn=11,indexer=annoy_index)

list(np.array(words)[:,0])

['wording',
 'specifying',
 'resolution',
 'spelled',
 'debated',
 'ratification',
 'timetable',
 'resolutions',
 'revise',
 'omitted',
 'timeframe']

In [22]:

print(trainData["comment_without_sw"][0])
not set(["negativ"]).isdisjoint(set(trainData["comment_without_sw"][0].split()))

things might negative frequency dependent selection going least common phenotype reflected genotype going advantage environment instance prey animal vole light dark phenotype predator might recognize common phenotype food light voles common foxes may keeping closer eye light phenotypic voles recognising good prey reduce light causing alleles due increased predation dark genotypes increase proportion population scenario reversed cycle continues perpetually nnhowever unlikely strictly yearly usually takes time year entire populations allele frequencies change enough make large enough difference alter fitness nnmore likely year year basis population experiencing fluctuating selection alternating conditions environment favor one genotype another perhaps plant species living area flooded every year two phenotypes population plants much better dryer year one better wet year flooding drytype genotype fitness leading offspring therefore dry alleles population however flooded years wetliking ph

False

In [23]:

def create_vector_from_comment(data):
  feature_vectors = []
  for comment in data['comment_without_sw']:
    vector = []
    for key in freq_sub.keys():
      words = [key]
      similar = None
      if key in model.vocab:
        sim = model.most_similar([model[key]], topn=11,indexer=annoy_index)
        similar = list(np.array(sim)[:,0])
      if(similar is not None):
          words += similar
      if not set(words).isdisjoint(set(comment.split())):
        vector.append(1)
      else:
        vector.append(0)
    feature_vectors.append(vector)

  return feature_vectors

In [24]:
feature_vectors = create_vector_from_comment(trainData)


13 min

In [25]:
fv = np.array(feature_vectors)

In [26]:
print(np.where(fv[10] == 1))
indices = np.where(fv[10] == 1)
print(freq_sub.iloc[indices])
print(trainData["Comment"][10])

words = model.most_similar("amino", topn=11,indexer=annoy_index)

list(np.array(words)[:,0])

(array([  4,  41,  87, 118, 278, 282, 536]),)
amino         62
bunch         29
labs          20
wild          17
titration     10
hospital      10
chloroform     6
dtype: int64
 Is H2S what produced when the egg protein is being denatured? Then titration with a basic solution like NaOH would allow me to determine the amount of H2S, hence the amount of denatured protein. Is my thinking correct? It's a school lab so I don't have access to other fancy equipments :/


['amino',
 'residues',
 'protein',
 'proteins',
 'conserved',
 'peptides',
 'pathway',
 'esters',
 'receptors',
 'specificity',
 'receptor']

In [27]:
fv.shape

(8695, 691)

In [28]:
len(freq_sub)

691

In [29]:
fv = np.array(feature_vectors)

Y_labels = trainData["Topic"].copy()

counter = 0
for x in fv:
  if np.where(x == 1)[0].shape[0] == 0:
    fv = np.delete(fv , x ,axis = 0)
    Y_labels = Y_labels.drop(counter)
  counter += 1
fv = fv.astype(np.float64)

In [226]:
len(fv)

7456

In [30]:
D = np.zeros((len(fv),len(fv)))

In [31]:
D_curr = np.zeros((len(fv),len(fv)))

In [238]:
g = 2500
K = 3
C_target = 3
C_previous = len(fv)


C_current = len(fv) // g 



Rk = np.zeros((len(fv) , K),dtype=int)

In [34]:
fv.shape

(7456, 691)

In [196]:
import numba

@numba.jit(nopython=True,parallel=True)
def calc_dist(vec1 , vec2):
  return np.linalg.norm(vec1-vec2)

In [200]:
def calc_D_orginal():
  for i in range(len(fv)):
    sum = 0;
    for j in range(i + 1, len(fv)):
      D[i][j] = calc_dist(fv[i],fv[j])
      D[j][i] = D[i][j]

In [201]:
calc_D_orginal()
# 3:33 min


In [203]:
def calc_Rk():
  for i in range(len(fv)):
    result = np.argpartition(D[i], K)
    Rk[i] = result[:K]



In [204]:
calc_Rk()


In [206]:
import numba

@numba.jit(nopython=True,parallel=True)
def D_current(i, j):
  sum = 0
  for index1 in Rk[i]:
    for index2 in Rk[j]:
      sum += D[index1][index2]
  return sum / math.pow(K+1,2)

In [237]:
#44 sec
for i in range(len(fv)):
  for j in range(i, len(fv)):
    D_curr[i][j] = D_current(i,j)
    D_curr[j][i] = D_curr[i][j] 

In [239]:
import numba

@numba.jit(nopython=True,parallel=True)
def find_first_key():
  min_dist = np.inf
  index = None
  for i in range(len(fv)):
    sum = 0
    for j in range(len(fv)):
      sum += D_curr[i][j]
    if sum < min_dist:
      min_dist = sum
      index = i
  return index

In [228]:
S_current = set()
# S_current.add(find_first_key())
S_current

{11}

In [49]:
!pip install funcy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [240]:
from funcy import print_durations

@numba.jit(nopython=True,parallel=True)
def improved_argmax(all_vals):
    return np.argmax(all_vals)

In [241]:
def find_rest_keys():

  n = C_current - 1

  all_vals = np.zeros(len(fv))
  for i in range(n):
    for j in range(len(fv)):
      for key in S_current:
        key_min = np.inf
        if (j not in S_current):
          distance = D_curr[key][j]
          if(distance < key_min):
            all_vals[j] = distance
            key_min = distance
    # idx = np.argmax(all_vals)
    S_current.add(improved_argmax(all_vals))
    all_vals = np.zeros(len(fv))



      

In [229]:
# find_rest_keys()

In [242]:
def find_keys():
  S_current.append(find_first_key())
  if C_target > 1:
    find_rest_keys()

array([   0,    1,    2, ..., 7453, 7454, 7455])

In [243]:
def find_label():
  for i in range(len(fv)):
    min = np.inf
    index = None
    if(L[i] not in S_current):
      for key in S_current:
        if D_curr[L[i]][key] < min:
          min = D_curr[L[i]][key]
          index = key
      L[i] = index

In [230]:
# L = np.arange(len(fv))
# MapL = []
# find_label()
# count = 0
# for i in set(L):
#   L = np.where(L == i,count,L)
#   count += 1

# for i in range(len(L)):
#   MapL.append(np.where(L  == i))
# print(MapL[3])
#dataIndex = np.arange(len(fv))

(array([6580]),)


In [231]:
S_current

{11, 369, 2232, 6473, 6580}

In [244]:
import numba

@numba.jit(nopython=True,parallel=True)
def calc_D_current(i,j,num1,num2):
  sum = 0
  for index1 in Rk[i]:
    for index2 in Rk[j]:
      sum += D[index1][index2]
  return sum / (num1*num2)

In [105]:
# P = np.zeros(C_current)
# for i in range(np.amin(L),np.amax(L)):
#    P[i] = np.count_nonzero(L == i)
# P

array([0., 0., 0., ..., 0., 0., 0.])

In [245]:
@numba.jit(nopython=True,parallel=True)
def calc_D_current(v1, v2):
  sum = 0
  for index1 in v1:
    for index2 in v2:
      sum += D[index1][index2]
  return sum / (len(v1)*len(v2))

In [233]:
# v1 = []
# v2 = []
# i = 1036
# j = 2
# v1.append(Rk[i])
# v2.append(Rk[j])
# if(L[i] != L[j]):
#   v1.append(MapL[L[i]])
#   v2.append(MapL[L[j]])
#   v1 = np.unique(np.concatenate((v1[0],v1[1][0]),0))
#   v2 = np.unique(np.concatenate((v2[0],v2[1][0]),0))


# print(v1)
# print(v2)
# print(MapL[L[i]])
# print(MapL[L[j]])

[array([0, 0, 0])]
[array([0, 0, 0])]
(array([   0,    1,    2, ..., 7453, 7454, 7455]),)
(array([   0,    1,    2, ..., 7453, 7454, 7455]),)


In [ ]:
# for i in range(len(fv)):
#   for j in range(len(fv)):
#     v1 = []
#     v2 = []
#     v1.append(Rk[i])
#     v2.append(Rk[j])
#     if(L[i] != L[j]):
#       v1.append(MapL[L[i]])
#       v2.append(MapL[L[j]])
#       v1 = np.unique(np.concatenate((v1[0],v1[1][0]),0))
#       v2 = np.unique(np.concatenate((v2[0],v2[1][0]),0))
#     else :
#       v1 = np.unique(np.concatenate((v1[0],[]),0)).astype(np.int64)
#       v2 = np.unique(np.concatenate((v2[0],[]),0)).astype(np.int64)
#     D_curr[i][j] = calc_D_current(v1, v2)
#     D_curr[i][j] = D_curr[j][i]
#   print(i)

In [ ]:
while(C_current > C_target):
  find_keys()
  find_label()
  count = 0
  for i in set(L):
    L = np.where(L == i,count,L)
    count += 1
  for i in range(len(L)):
    MapL.append(np.where(L  == i))
  

  for i in range(len(fv)):
    for j in range(len(fv)):
      v1 = []
      v2 = []
      v1.append(Rk[i])
      v2.append(Rk[j])
      if(L[i] != L[j]):
        v1.append(MapL[L[i]])
        v2.append(MapL[L[j]])
        v1 = np.unique(np.concatenate((v1[0],v1[1][0]),0))
        v2 = np.unique(np.concatenate((v2[0],v2[1][0]),0))
      D_curr[i][j] = calc_D_current(v1, v2)
      D_curr[i][j] = D_curr[j][i]
  C_current = C_current // g

In [ ]:
find_keys()
find_label()
count = 0
for i in set(L):
  L = np.where(L == i,count,L)
  count += 1
print(L) #clustered

In [247]:
test_feature_vectors = create_vector_from_comment(testData)
fv_test = np.array(test_feature_vectors)

In [ ]:
for i in range(len(fv_test)):
  correct = 0
  min_dist = np.inf
  index = None
  for key in S_current:
    distance = calc_dist(fv_test[i],key)
    if distance < min:
      min = distance
      index = key
  if(testData['comment_without_sw'][i].Topic == trainData['comment_without_sw'][index].Topic):
    correct += 1

print(correct/(len(fv_test))) #accuracy